<a href="https://colab.research.google.com/github/google-gemini/workshops/blob/main/adventure/Dungeon_Adventure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install --q crewai langchain_google_genai > /dev/null

In [59]:
import os
import random
from textwrap import dedent
from typing import List

from crewai import LLM, Agent, Crew, Task
from google.colab import userdata
from IPython.display import Markdown, display
from pydantic import BaseModel

In [62]:
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

gemini = LLM(model="gemini/gemini-1.5-pro-latest")


def make_dungeon_master_agent(llm) -> Agent:
    """
    Create a Dungeon Master agent for a text-based adventure game.

    Args:
        llm (LLM): The language model instance to use for narration and decision-making.

    Returns:
        Agent: Configured Dungeon Master agent with relevant roles and goals.
    """
    role = "Dungeon Master (DM)"
    goal = (
        "Guide the adventurers through an evolving story by narrating scenes, "
        "evaluating player actions based on dice rolls, and introducing new challenges."
    )
    backstory = (
        "You are the Dungeon Master, an omniscient storyteller controlling the "
        "fantasy world. You describe vivid scenes, determine the outcomes of player actions, "
        "and keep the story moving forward. You ensure a fair and engaging experience, "
        "balancing difficulty and excitement for the players."
    )

    return Agent(llm=llm, role=role, goal=goal, backstory=backstory, verbose=True)


def make_dungeon_master_task(agent: Agent) -> Task:
    """
    Create a Dungeon Master task that returns concise text describing events and the next scenario.
    The task ensures brevity and only responds to actual player actions.

    Args:
        agent (Agent): The Dungeon Master agent.

    Returns:
        Task: A CrewAI task for processing player actions and continuing the story.
    """

    task = Task(
        description=dedent(
            """\
            You are the Dungeon Master, guiding the players through an adventure.

            Game Context:
            {game_history}

            Players in the game:
            {player_roles}

            Instructions:
            1. The only characters who exist in the game are in the list above. Do not invent new characters.
            2. Only respond to actions that were actually taken—do not create additional actions.
            3. Be concise: Your response should be no more than 3 sentences describing the results of the turn.
            4. Advance the story by setting up the next scenario in 1 or 2 sentences.
            5. Do not include unnecessary details—keep responses short and clear.

            Expected Output:
            The output should be a single block of plain text. The first part describes the outcome of the current turn in 3 sentences or fewer, and the second part sets up the next scenario in 1 or 2 sentences.
            """
        ),
        agent=agent,
        expected_output="A short block of text (3 sentences for results, 1-2 sentences for next scenario).",
    )

    return task


def make_player_task(agent: Agent) -> Task:
    """
    Create a player task to decide the next move based on the current scenario.
    The task explicitly delegates action selection to the specified agent.

    Args:
        agent (Agent): The player agent making the decision.

    Returns:
        Task: A CrewAI task for selecting an action.
    """

    task = Task(
        description=dedent(
            """\
            You are a player in a fantasy adventure game. Your goal is to react appropriately to the current scenario.

            Current Scenario:
            {current_scenario}

            Instructions:
            1. You are {current_player}. You must take an action this turn.
            2. Consider your role, abilities, and the situation at hand.
            3. Choose a single action that best suits your character.
            4. Describe your action in the third person, starting with your name.
               Example: "Thorn swings his sword at the goblin."
               Example: "Lyra quietly sneaks past the guards."

            Expected Output:
            The output should be a single line of text describing the player's action in the third person, starting with their name.
            """
        ),
        agent=agent,
        expected_output="A single line of text describing the player's action in the third person, starting with their name.",
    )

    return task


# --- Dungeon Master Crew ---
def make_dungeon_master_crew(llm):
    """Creates the DM Crew with a single Dungeon Master agent."""
    dm_agent = make_dungeon_master_agent(llm)
    dm_task = make_dungeon_master_task(dm_agent)

    return Crew(agents=[dm_agent], tasks=[dm_task])


# --- Player Crew ---


def make_player_crew(player_agents):
    """
    Creates a CrewAI Player Crew from a list of player agents.
    Each agent is assigned a corresponding player task.

    Args:
        player_agents (list[Agent]): List of CrewAI player agents.

    Returns:
        Crew: A CrewAI crew with the player agents and their tasks.
    """
    player_tasks = [make_player_task(agent) for agent in player_agents]

    return Crew(agents=player_agents, tasks=player_tasks)


# --- Game State ---


class GameState:
    """Stores game progression including campaign, turns, and current scenario."""

    def __init__(self, campaign_description):
        self.campaign_description = campaign_description
        self.turns = []
        self.current_scenario = campaign_description  # Initial world setup

    def add_turn(self, player_actions, next_scenario):
        """Records a turn and updates the game state."""
        self.turns.append({"player_actions": player_actions, "scenario": next_scenario})
        self.current_scenario = next_scenario  # Advance to new scenario


def format_game_state(game_state):
    """
    Formats the game state into a structured Markdown output.

    Args:
        game_state (GameState): The current game state containing campaign details and turns.

    Returns:
        None: Displays formatted Markdown output in Colab.
    """
    md_output = f"## 🏰 Adventure Log\n\n"
    md_output += f"### Campaign: {game_state.campaign_description}\n\n"

    for i, turn in enumerate(game_state.turns, 1):
        md_output += f"### 🔄 Turn {i}\n\n"
        md_output += f"**Scenario:**\n\n{turn['scenario']}\n\n"
        md_output += "**🎭 Player Actions:**\n"

        for action in turn["player_actions"]:
            md_output += f"- {action}\n"

        md_output += "\n---\n\n"

    display(Markdown(md_output))


# --- Game Loop ---


def run_game_loop(llm, game_state, player_agents):
    """Runs the DM Crew → Player Crew → Updates State, using a random turn order."""
    dm_crew = make_dungeon_master_crew(llm)

    # Extract only the roles of real players
    player_roles = [agent.role for agent in player_agents]

    while True:
        # 1. Run the DM Crew to generate the next scenario
        next_scenario = dm_crew.kickoff(inputs={"game_history": game_state.turns, "player_roles": player_roles})

        # 2. Randomly select a player agent to act
        current_player = random.choice(player_agents)
        player_crew = make_player_crew([current_player])
        player_moves = player_crew.kickoff(
            inputs={"current_scenario": next_scenario.raw, "current_player": current_player.role}
        )

        # 3. Update Game State
        game_state.add_turn([player_moves.raw], next_scenario.raw)

        # 4. Repeat (or break based on conditions)
        if len(game_state.turns) >= 10:  # Stop after 10 turns for now
            print("Game Over!")
            break


# Define Warrior Agent
warrior = Agent(
    llm=gemini,
    role="Warrior Poet (Aspiring)",
    goal="Slay foes with both sword and verse, and find inspiration for their epic poem.",
    backstory="A warrior who believes that true strength lies in both might and words. They carry a worn notebook filled with their attempts at poetry, which they often recite during or after battles.",
    verbose=True,
)

# Define Thief Agent
monk = Agent(
    llm=gemini,
    role="Joyful Acrobat Monk",
    goal="Spread joy and enlightenment through movement and laughter, using martial arts as a form of expression.",
    backstory="A monk who embraces the lighter side of life, finding enlightenment in playful movement and laughter. They believe that joy is a path to inner peace.",
    verbose=True,
)

paladin = Agent(
    llm=gemini,
    role="Stoic Guardian Paladin",
    goal="Uphold the sacred oaths and protect the realm from darkness, maintaining unwavering resolve in the face of adversity.",
    backstory="A paladin of few words, their actions speaking louder than any pronouncements. They are a pillar of strength and stability, unwavering in their duty to protect the realm from the forces of darkness.",
    verbose=True,
)

# List of player agents
player_agents = [warrior, monk, paladin]

game_state = GameState("You arrive at an ancient fortress, its gates wide open.")
run_game_loop(gemini, game_state, player_agents)
format_game_state(game_state)

# Agent: Dungeon Master (DM)
## Task: You are the Dungeon Master, guiding the players through an adventure.

Game Context:
[]

Players in the game:
['Warrior Poet (Aspiring)', 'Joyful Acrobat Monk', 'Stoic Guardian Paladin']

Instructions:
1. The only characters who exist in the game are in the list above. Do not invent new characters.
2. Only respond to actions that were actually taken—do not create additional actions.
3. Be concise: Your response should be no more than 3 sentences describing the results of the turn.
4. Advance the story by setting up the next scenario in 1 or 2 sentences.
5. Do not include unnecessary details—keep responses short and clear.

Expected Output:
The output should be a single block of plain text. The first part describes the outcome of the current turn in 3 sentences or fewer, and the second part sets up the next scenario in 1 or 2 sentences.



# Agent: Dungeon Master (DM)
## Final Answer: 
The Warrior Poet, Joyful Acrobat Monk, and Stoic Guardian Paladi

## 🏰 Adventure Log

### Campaign: You arrive at an ancient fortress, its gates wide open.

### 🔄 Turn 1

**Scenario:**

The Warrior Poet, Joyful Acrobat Monk, and Stoic Guardian Paladin find themselves at the edge of a whispering forest, an unnatural stillness hanging in the air.  A faint trail leads into the dense trees, seemingly the only path forward. Will the adventurers brave the unknown depths of the whispering woods?

**🎭 Player Actions:**
- The Warrior Poet unsheathes their sword, its polished surface glinting in the muted light, and steps forward, reciting, "Into the silent wood we stride, where whispers dance and shadows hide..."

---

### 🔄 Turn 2

**Scenario:**

As the Warrior Poet's words echo through the silent wood, a flock of ravens bursts from the trees, their harsh cries breaking the unnatural quiet.  The trail leads deeper into the woods, beckoning the adventurers forward.  A sense of unease settles upon the group as the ravens circle overhead.  Will the Joyful Acrobat Monk and Stoic Guardian Paladin follow the Warrior Poet deeper into the whispering woods, or will they seek another path?

**🎭 Player Actions:**
- The Joyful Acrobat Monk cartwheels forward, chuckling. "Come, friends! The ravens guide us! Let's see what wonders the whispering woods hold! Perhaps we'll find enlightenment in the rustling leaves and chirping birds!"

---

### 🔄 Turn 3

**Scenario:**

The ravens' cries seem to mock the adventurers' bravery as they venture deeper into the whispering woods.  The unnatural stillness returns, broken only by the rustling of unseen things in the undergrowth.  The faint trail leads them to a clearing, where a crumbling stone altar stands draped in moss.  Etched into the altar is a strange symbol, pulsing with a faint, ethereal light. Will the adventurers investigate the altar, or will they heed the warning of the whispering woods and turn back?

**🎭 Player Actions:**
- Warrior Poet (Aspiring) approaches the altar cautiously, hand resting on the hilt of his sword, and recites in a low voice, "O cryptic stone, what secrets do you keep?  Speak to me in whispers, not in screams, lest my blade sing a song of sorrow."

---

### 🔄 Turn 4

**Scenario:**

As the Warrior Poet speaks, the symbol on the altar glows brighter, and a low hum fills the air.  From the shadows, three ghostly figures materialize, their eyes fixed on the adventurers.  They seem to be guardians of the altar, their spectral forms radiating an ancient power. The ghostly figures raise their arms, and spectral weapons shimmer into existence. Will the adventurers fight or flee?

**🎭 Player Actions:**
- Warrior Poet raises his sword, a defiant gleam in his eye, and bellows, "Guardians of ages past, your slumber is disturbed! Though I respect your vigil, my quest demands I pass. Prepare to face the might of my blade...and the sting of my verse!" He then recites, voice ringing with power,  "From shadows deep you rise to claim, this hallowed ground, your spectral game. But destiny's call, I can't ignore, so steel yourselves for poetic war!"

---

### 🔄 Turn 5

**Scenario:**

The ghostly guardians, enraged by the Warrior Poet's challenge, lunge forward, their spectral weapons flashing.  The battle begins, the air thick with the clash of steel and the hum of otherworldly energy.  The Joyful Acrobat Monk and Stoic Guardian Paladin must now decide how to support their companion.  Will they join the fray, or attempt to find another way to appease the ghostly guardians and continue their quest?

**🎭 Player Actions:**
- The Warrior Poet deflects a spectral blow, then shouts, "Though shadows cling and spirits moan, my verse will break these walls of bone!" He swings his sword, aiming not to destroy the ghostly guardians, but to carve a path through their ethereal forms while reciting a piece of his epic poem-in-progress.  "From dust we rise, to dust return, but courage burns, a fiery kern!"

---

### 🔄 Turn 6

**Scenario:**

The Warrior Poet's sword, imbued with poetic power, carves shimmering trails through the spectral forms, momentarily disrupting their attack but not harming them. The ghostly guardians, their forms flickering, retaliate with renewed fury, their spectral weapons radiating an icy chill.  The chilling touch weakens the Warrior Poet, slowing his movements.  The Joyful Acrobat Monk and Stoic Guardian Paladin see an opening to aid their companion and potentially discover a way to pacify the spirits. Do they press the attack, seek a peaceful resolution, or find another way to bypass the ghostly guardians and reach their ultimate goal?

**🎭 Player Actions:**
- The Joyful Acrobat Monk cartwheels between the Warrior Poet and the spectral guardians, letting out a peel of laughter. "Woe is me! Such serious faces for such a playful dance! Come, spirits, let's find some joy in this spectral shindig!" He then performs a series of acrobatic flips and spins, dodging the spectral weapons while attempting to disarm them with playful tugs and misdirections.

---

### 🔄 Turn 7

**Scenario:**

The Joyful Acrobat Monk's acrobatic maneuvers distract the ghostly guardians, creating an opening.  The spectral weapons pass harmlessly through the monk's agile form, their chilling touch unable to grasp him.  The Warrior Poet, though weakened, sees an opportunity to press the attack.  

The Stoic Guardian Paladin must now decide how to act: join the fight, seek a peaceful resolution, or discover another way past the guardians. What will the Paladin do?

**🎭 Player Actions:**
- Stoic Guardian Paladin strides forward, shield raised to protect the Warrior Poet, creating a bulwark against the ghostly guardians' attacks.

---

### 🔄 Turn 8

**Scenario:**

The Stoic Guardian Paladin's shield deflects the spectral blows, momentarily shielding the Warrior Poet from harm. The Joyful Acrobat Monk's distracting antics continue to disrupt the guardians, creating an opening for a decisive action.  The Warrior Poet, though weakened, finds renewed strength in the Paladin's protection.  The ghostly guardians, their forms flickering and unstable, seem vulnerable. Now, the adventurers must decide their next move: press the attack, seek a peaceful resolution, or attempt to bypass the guardians entirely. What will the Warrior Poet, Joyful Acrobat Monk, and Stoic Guardian Paladin do?

**🎭 Player Actions:**
- Stoic Guardian Paladin plants their shield firmly, creating a bulwark against the spectral guardians and shouts, "Strike now, while they are weakened!"

---

### 🔄 Turn 9

**Scenario:**

The Paladin's shield protects the Warrior Poet, allowing them to land a powerful blow, while the Monk's distractions keep the guardians off balance.  The combined efforts weaken the ghostly figures considerably, their forms flickering and fading. One of the guardians collapses into shimmering dust, its spectral weapon disappearing. The remaining two guardians, their forms now significantly fainter, retreat further into the shadows surrounding the altar.  Will the adventurers pursue the weakened guardians or investigate the altar now that it is more accessible?

**🎭 Player Actions:**
- The Warrior Poet, invigorated by the clash, shouts, "Flee not, wraiths! For your demise shall fuel my verse!" and advances towards the retreating guardians, notebook flapping open in his off-hand.

---

### 🔄 Turn 10

**Scenario:**

The Warrior Poet's advance pressures the remaining guardians, who dissipate into wisps of shadow, leaving the altar unguarded.  As the ghostly guardians vanish, the altar's inscription glows brightly, revealing a hidden compartment beneath. Will the adventurers investigate the compartment or continue their exploration of the whispering woods?

**🎭 Player Actions:**
- Joyful Acrobat Monk cartwheels towards the altar, exclaiming, "Ooooh! Shiny! What treasures await us within?"

---

